In [55]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from opencage.geocoder import OpenCageGeocode
from geopy.exc import GeocoderTimedOut
import folium
import json


In [2]:
source = requests.get("http://worldpopulationreview.com/continents/cities-in-europe/").text

In [54]:
geolocator = OpenCageGeocode('ea9636898f12432caccd5d49e4a11ec9')
def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [56]:
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
data = []
for row in table.findAll('tr'):
    temp = []
    for element in row.findAll('td'):
        temp.append(element.text)
    location = do_geocode('{}, {}'.format(temp[0], temp[1]))
    temp.append(location[0]['geometry']['lat'])
    temp.append(location[0]['geometry']['lng'])
    data.append(temp)
df = pd.DataFrame(data, columns = ['City', 'Country', 'Population', 'Latitude', 'Longitude'])

In [59]:
df.to_csv('my_data.csv')

In [82]:
df.head(10)

,City,Country,Population,Latitude,Longitude
0,Moscow,Russia,"10,381,222",55.750446,37.617494
1,London,United Kingdom,"7,556,900",51.507322,-0.127647
2,Saint Petersburg,Russia,"5,028,000",59.938732,30.316229
3,Berlin,Germany,"3,426,354",52.517037,13.388860
4,Madrid,Spain,"3,255,944",40.416705,-3.703582
5,Kyiv,Ukraine,"2,797,553",50.450034,30.524136
6,Rome,Italy,"2,318,895",41.894802,12.485338
7,Paris,France,"2,138,551",48.856697,2.351462
8,Bucharest,Romania,"1,877,155",44.436141,26.102720
9,Minsk,Belarus,"1,742,124",53.902334,27.561879


In [68]:
latitude = 55.4649
longitude = 65.3054
map_europe = folium.Map(location=[latitude, longitude], zoom_start=3)

for lat, lng, country, city in zip(df['Latitude'], df['Longitude'], df['Country'], df['City']):
    label = '{}, {}'.format(city, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        fill_color='##3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_europe)  
    
map_europe

In [138]:
Creds = pd.read_csv('FourSquareCredentials.csv')
CLIENT_ID = 'Creds['CLIENT_ID'][0]' # your Foursquare ID
CLIENT_SECRET = Creds['CLIENT_SECRET'][0] # your Foursquare Secret
VERSION = Creds['VERSION'][0] # Foursquare API version

In [84]:
radius = 10000
LIMIT = 100
def getNearbyVenues(names, countries, latitudes, longitudes):
    
    venues_list=[]
    for name, country, lat, lng in zip(names, countries, latitudes, longitudes):
        print('{}, {}'.format(name, country))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            country,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Country',           
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
europe_venues = pd.read_csv('europe_venues.csv')

In [155]:
start = 0

In [149]:
parts = list(range(50, df.shape[0] + 1, 50))    

In [152]:
for end in parts:
    temp = df[start:end]
    temp_venues = getNearbyVenues(names=temp['City'],
                                   countries=temp['Country'],
                                   latitudes=temp['Latitude'],
                                   longitudes=temp['Longitude']
                                  )
    europe_venues = europe_venues.append(temp_venues, ignore_index = True)
    start = end
    print(start)

Torun, Poland
Kielce, Poland
Jerez de la Frontera, Spain
Oradea, Romania
Sabadell, Spain
Mostoles, Spain
Linz, Austria
Alcala de Henares, Spain
Ivano-Frankivsk, Ukraine
Debrecen, Hungary
Padova, Italy
Erfurt, Germany
Vasyl'evsky Ostrov, Russia
Tampere, Finland
Harburg, Germany
Pskov, Russia
Favoriten, Austria
Nicosia, Cyprus
Charleroi, Belgium
Severnyy, Russia
Tilburg, Netherlands
Balakovo, Russia
Armavir, Russia
Bila Tserkva, Ukraine
Dudley, United Kingdom
Hagen, Germany
Gliwice, Poland
Pamplona, Spain
Rostock, Germany
Fuenlabrada, Spain
Aberdeen, United Kingdom
Reims, France
Engel's, Russia
Burgas, Bulgaria
Kassel, Germany
Severodvinsk, Russia
Portsmouth, United Kingdom
Newcastle upon Tyne, United Kingdom
Klaipeda, Lithuania
Zabrze, Poland
Zlatoust, Russia
Cork, Ireland
Vantaa, Finland
Syzran', Russia
Bytom, Poland
Almeria, Spain
Praga Poludnie, Poland
Sutton, United Kingdom
Petropavlovsk-Kamchatsky, Russia
Trieste, Italy


C:\Users\soumy\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


400
Donaustadt, Austria
Leganes, Spain
Le Havre, France
Swindon, United Kingdom
Dun Laoghaire, Ireland
San Sebastian, Spain
Hamm, Germany
Mainz, Germany
Brescia, Italy
Geneve, Switzerland
Cergy-Pontoise, France
Korolyov, Russia
Sants-Montjuic, Spain
Santander, Spain
Liege, Belgium
Kamensk-Ural'skiy, Russia
Prato, Italy
Saarbruecken, Germany
Groningen, Netherlands
Taranto, Italy
Crawley, United Kingdom
Castello de la Plana, Spain
Yasenevo, Russia
Podolsk, Russia
Burgos, Spain
Amadora, Portugal
Ipswich, United Kingdom
Bielsko-Biala, Poland
Yuzhno-Sakhalinsk, Russia
Almere Stad, Netherlands
Split, Croatia
Saint-Etienne, France
Turku, Finland
Wigan, United Kingdom
Kramators'k, Ukraine
Croydon, United Kingdom
Miskolc, Hungary
Walsall, United Kingdom
Herne, Germany
Lyublino, Russia
Mansfield, United Kingdom
Olsztyn, Poland
Bacau, Romania
Oxford, United Kingdom
Muelheim, Germany
Albacete, Spain
Reggio Calabria, Italy
Arad, Romania
Baranovichi, Belarus
Toulon, France
450
Angers, France
Horta-G

In [161]:
europe_venues.columns

Index(['City', 'City Latitude', 'City Longitude', 'Country', 'Venue',
       'Venue Category', 'Venue Latitude', 'Venue Longitude'],
      dtype='object')

In [160]:
europe_venues.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)

In [162]:
europe_venues.to_csv('europe_venues.csv')